# Inserting Data Without Duplicates

Here we test inserting data without duplicates. Afterwards we'll test for missing data inside of the databases.

In [1]:
%pwd

'/home/skywalker/PycharmProjects/jamboree/test/notebooks'

In [2]:
%cd ../..

/home/skywalker/PycharmProjects/jamboree


In [3]:
from jamboree import Jamboree
import pandas as pd
import datetime
import pandas_datareader.data as web
from pandas import Series, DataFrame

In [4]:
from maya import MayaDT
import maya
import copy

In [5]:
import random
import orjson

In [6]:
from typing import List, Dict, Any

In [7]:
jam_session = Jamboree()

Unable to create library with name: events


LibraryNotFoundException: Library events was not correctly initialized in <Arctic at 0x7efbaa926128, connected to MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True, maxpoolsize=4, sockettimeoutms=600000, connecttimeoutms=2000, serverselectiontimeoutms=30000)>.
Reason: ServerSelectionTimeoutError('localhost:27017: [Errno 111] Connection refused',))

In [ ]:
start = datetime.datetime(1986, 3, 14)
end = datetime.datetime(2020, 1, 6)

In [ ]:
apple_df = web.DataReader("AAPL", 'yahoo', start, end)
msft_df = web.DataReader("MSFT", 'yahoo', start, end)

In [ ]:
apple_df

In [ ]:
def get_year_month_day(time:MayaDT):
    print(f"{time.day}-{time.month}-{time.year}")

In [ ]:
def get_time_dt(df):
    indexes = df.index
    indexes = [maya.MayaDT.from_datetime(index.to_pydatetime()) for index in indexes]
    return indexes

In [ ]:
def df_records(df):
    return df.to_dict("records")

In [ ]:
def standardize_record(record):
    closing_record = {}
    if "Close" in record:
        closing_record['close'] = record["Close"]
    if "Open" in record:
        closing_record['open'] = record["Open"]
    if "Low" in record:
        closing_record['low'] = record["Low"]
    if "High" in record:
        closing_record['high'] = record["High"]
    if "Volume" in record:
        closing_record['volume'] = record["Volume"]
    if "Adj Close" in record:
        closing_record['adj_close'] = record[""]
    return closing_record

In [ ]:
def standardize_outputs(records:List[Dict[str, Any]]):
    if len(records) == 0:
        return []
    _records = [standardize_record(rec) for rec in records]
    return _records

In [ ]:
def add_time(records, times):
    if len(records) == 0 or (len(records) != len(times)):
        return []
    
    _records = []
    for index, rec in enumerate(records):
        rec['time'] = times[index]._epoch
        _records.append(rec)
    return _records

In [ ]:
def teardown(df):
    """Breaks the dataframe into a bunch of dictionaries"""
    indexes = get_time_dt(df)
    records = df_records(df)
    standardized = standardize_outputs(records)
#     print(standardized)
    with_time = add_time(standardized, indexes)
    return with_time

In [ ]:
dt_time = teardown(apple_df)

In [ ]:
def flip(n=0.05):
    if random.uniform(0, 1) < n:
        return True
    return False

In [ ]:
def create_duplicates(frame_dict_list:List[Dict]):
    if len(frame_dict_list) == 0:
        return []
    
    final_list = []
    for item in frame_dict_list:
        final_list.append(item)
        if flip(0.1):
            final_list.append(item)
    return final_list

In [ ]:
# len(dups)

In [ ]:
last_200 = dt_time[-200:]
last_300 = dt_time[-300:]
last_200_dups = create_duplicates(last_200)
last_300_dups = create_duplicates(last_300)

In [ ]:
upsert_data_one = jam_session.bulk_upsert_redis({"type": "sample_save", "asset": "AAPL", "label": "duplication"}, last_200)
upsert_data_two = jam_session.bulk_upsert_redis({"type": "sample_save", "asset": "AAPL", "label": "duplication"}, last_300)
upsert_data_one_dups = jam_session.bulk_upsert_redis({"type": "sample_save", "asset": "AAPL", "label": "duplication"}, last_200_dups)
upsert_data_two_dups = jam_session.bulk_upsert_redis({"type": "sample_save", "asset": "AAPL", "label": "duplication"}, last_300_dups)

In [ ]:
main_hash = upsert_data_one.get("hash")

In [ ]:
up1 = upsert_data_one.get('updated', [])
up2 = upsert_data_two.get('updated', [])
up3 = upsert_data_one_dups.get('updated', [])
up4 = upsert_data_two_dups.get('updated', [])

In [ ]:
cr1 = [orjson.dumps(x) for x in up1]
cr2 = [orjson.dumps(x) for x in up2]
cr3 = [orjson.dumps(x) for x in up3]
cr4 = [orjson.dumps(x) for x in up4]

In [ ]:
set1 = set(cr1)
set2 = set(cr2)
set3 = set(cr3)
set4 = set(cr4)

In [ ]:
print(len(set1))
print(len(set2))
print(len(set3))
print(len(set4))

In [ ]:
jam_session.redis.sadd(set_key, *set(cr3))

In [ ]:
def deserialize_list(serialized_list:list):
    if len(serialized_list) == 0:
        return []
    
    return [orjson.loads(x) for x in serialized_list]

In [ ]:
def add_timestamp(item):
    item['timestamp'] = maya.now()._epoch
    return item

In [ ]:
def get_addable_items(set_key, added_set):
    existing = set(jam_session.redis.smembers(set_key))
    addable_items = set(set2 - existing)
    if len(addable_items) == 0:
        return []
    listified = list(addable_items)
    deku = deserialize_list(listified)
    timestamped = [add_timestamp(x) for x in deku]
    return timestamped

In [ ]:
# updated_set = set(serialized_updated)

In [ ]:
get_addable_items(set_key, set2)

In [ ]:
# jam_session.redis.smembers(set_key, 0, -1)

In [ ]:
set(retrieved - updated_set)

In [ ]:
len(retrieved)

In [ ]:
len(updated_set)